# Image set preparation

This notebook illustrates how the image set used for training and evaluation was generated from freely available aerial imagery and land use datasets.

## Outline
- [Source data](#sourcedata)
   - [National Land Cover Database](#nlcd)
   - [National Agriculture Imagery Program](#naip)
- [Preparing an Azure Data Science Virtual Machine for image extraction](#dsvm)
   - [Provisioning and accessing the DSVM](#provision)
   - [Installing Python packages and supporting software](#install)
   - [Downloading input data locally](#download)
- [Producing image sets for training and evaluation](#production)
   - [Converting NAIP images from MrSID to GeoTIFF](#conversion)
   - [Add tiling for fast data extraction](#tiling)
   - [Find a lat-lon bounding box for the region](#box)
   - [Get the approximate dimensions of the bounding box in meters](#meters)
   - [Finding regions with consistent land use class](#consistent)
   - [Extract and save images for tiles of interest](#extraction)
   - [Automating extraction from multiple counties](#automation)

<a name="sourcedata"></a>
## Source data

<a name="naip"></a>
### National Agriculture Imagery Program

The US [National Agriculture Imagery Program](https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/index) is run by the [US Department of Agriculture](https://www.usda.gov/)'s [Farm Service Agency](https://www.fsa.usda.gov/). NAIP images are provided within one month of image collection, in natural (RGB) color with 1-meter ground sample distance. In this tutorial, we use Compressed County Mosaics obtained from the [Geospatial Data Gateway](https://gdg.sc.egov.usda.gov/).

<a name="nlcd"></a>
### National Land Cover Database

The US [National Land Cover Database](https://www.mrlc.gov/nlcd2011.php) (NLCD), maintained by the [Multi-Resolution Land Characteristic Consortium](https://www.mrlc.gov/), provides land use labels at 30-meter spatial resolution for the United States. The [sixteen land use classes](https://www.mrlc.gov/nlcd11_leg.php) in the NLCD are organized hierarchically into major (Developed, Forested, Planted/Cultivated, &c.) and minor (e.g. Deciduous, Evergreen, or Mixed Forest) categories. Classifications are based largely on seasonally-collected satellite imagery (including data collected outside the visual spectrum). Because of the extensive post-processing required to prepare this dataset, new versions are published approximately every five years, often with a multi-year delay between data collection and publication. For this project, we use the [NLCD 2011 Land Cover](https://www.mrlc.gov/nlcd11_data.php) dataset.

For more information on the NLCD, please see the following publication:

Homer, C.G., Dewitz, J.A., Yang, L., Jin, S., Danielson, P., Xian, G., Coulston, J., Herold, N.D., Wickham, J.D., and Megown, K., 2015, [Completion of the 2011 National Land Cover Database for the conterminous United States - Representing a decade of land cover change information](http://bit.ly/1K7WjO3). Photogrammetric Engineering and Remote Sensing, v. 81, no. 5, p. 345-354 

<a name="dsvm"></a>
## Preparing an Azure Data Science Virtual Machine for image extraction

For reproducibility and brevity, this walkthrough describes how to set up an [Azure Data Science Virtual Machine](https://azure.microsoft.com/en-us/marketplace/partners/microsoft-ads/standard-data-science-vm/) for image extraction. It is very likely that the steps below can be adapted for your own computer.

<a name="provision"></a>
### Provisioning and accessing the DSVM

To provision a Data Science Virtual Machine, you will need an [Azure account](https://azure.microsoft.com/en-us/free/). Click the "Deploy to Azure" button to begin the process:

<a href="https://azuredeploy.net/?repository=https://github.com/Azure/Azure-MachineLearning-DataScience/tree/master/Data-Science-Virtual-Machine/Windows">![Deploy to Azure](https://camo.githubusercontent.com/a941ea1d057c4efc2dcc0a680f43c97728ec0bd8/687474703a2f2f617a7572656465706c6f792e6e65742f6465706c6f79627574746f6e2e737667)</a>

After logging in, you can select a name for your VM, login information, and [VM size](https://docs.microsoft.com/en-us/azure/virtual-machines/virtual-machines-windows-sizes#a-series). We chose an "A4" VM because of the large temporary hard disk space included. (Aerial imagery data for some counties can take up >100 GB when decompressed and converted to GeoTIFF format.) If your hard disk requirements are smaller, you may prefer a VM with a solid-state drive for faster file I/O. Confirm your selections to begin deployment.

After deployment concludes, navigate to the VM's overview pane in [Azure Portal](https://portal.azure.com) by e.g. searching for the VM or resource name. Click the "Connect" button along the top of the pane to download a Remote Desktop connection file. Once the download completes, double-click the file to connect to your VM.

Note that your VM will continue to run even after you disconnect from Remote Desktop. You can stop or delete the VM at any time from [Azure Portal](https://portal.azure.com).

<a name="install"></a>
### Installing Python packages and supporting software

#### LizardTech's GeoExpress Command Line Applications
NAIP imagery provided in [MrSID](https://en.wikipedia.org/wiki/MrSID) format: we will convert them to GeoTIFF format using LizardTech's free [GeoExpress Command Line Applications](https://www.lizardtech.com/gis-tools/tools-and-utilities). After downloading and decompressing the archive on your VM, copy the files to a permanent directory of your choice. Add the path to the binary file `mrsidgeodecode` here:

In [ ]:
mrsidgeodecode_path = 'C:\\Program Files\\GeoExpressCLUtils-9.5.0.4326-win64\\bin\\mrsidgeodecode'

#### Python packages
To read the resulting GeoTIFF, we will use the GDAL package from within Python. As of this writing, Christoph Gohlke's [Unofficial Windows Binaries for Python Extension Packages](http://www.lfd.uci.edu/~gohlke/pythonlibs/) provide the easiest means to install the GDAL package, binaries, and library. We downloaded the Python 3.5 wheels for the following packages from Christoph's site for this walkthrough:
- [GDAL](http://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal)
- [Basemap](http://www.lfd.uci.edu/~gohlke/pythonlibs/#basemap)
- [Fiona](http://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona)
- [Shapely](http://www.lfd.uci.edu/~gohlke/pythonlibs/#shapely)

To install these wheels:
1. Launch an Anaconda Prompt
1. Activate the Python 3.5 environment pre-installed on your DSVM using the following command:

  `activate py35`<br/><br/>
  
1. Install each wheel (`[wheel filename]`) with the following command:

  `pip install [wheel filename]`

<a name="download"></a>
### Downloading input data locally

#### National Land Cover Database

[Download](https://www.mrlc.gov/nlcd11_data.php) the compressed 2011 National Land Cover Database file (1.1 GB) and extract its contents to a folder of your choice (we used D:\nlcd). Store the filepath with a `.img` extension by editing and executing the cell below:

In [ ]:
nlcd_filepath = 'D:\\nlcd\\nlcd_2011_landcover_2011_edition_2014_10_10.img'

#### National Agriculture Imagery Program

As of this writing (1/31/2017), an unplanned outage complicates access to NAIP Compressed County Mosaics via the [Geospatial Data Gateway](https://gdg.sc.egov.usda.gov/). We have mirrored [a sample file](https://mawahstorage.blob.core.windows.net/aerialimageclassification/naipsample/ortho_imagery_NAIPM16_ma017_3344056_01.zip) (Middlesex County, MA 2016 data) for your convenience in following this tutorial. (You can make additional county requests using the email address provided on the gateway.) Download and decompress the file, storing the folder path in the following variable:

In [ ]:
naip_dir = 'D:\\naip\\ortho_imagery_NAIPM16_ma017_3344056_01'

<a name="production"></a>
## Producing image sets for training and evaluation

<a name="conversion"></a>
### Converting NAIP images from MrSID to GeoTIFF

We will use LizardTech's GeoExpress Command Line Application `mrsidgeodecode` to convert the aerial imagery data from MrSID format to GeoTIFF. Expect a >10-fold expansion in file size during conversion.

In [ ]:
import os
import glob
import shutil

filename_base = None
for filename in os.listdir(naip_dir):
    if filename.endswith('.sid'):
        filename_base = filename.split('.sid')[0]
        break
if filename_base == None:
    raise Exception("Couldn't find the MrSID file in {}".format(naip_dir))

geodecode_command = [mrsidgeodecode_path, '-wf',
                     '-i', os.path.join(naip_dir, '{}.sid'.format(filename_base)),
                     '-o', os.path.join(naip_dir, '{}.tif'.format(filename_base))]
results = shutil.run(geodecode_command)

<a name="tiling"></a>
### Add tiling for fast data extraction

To speed up image extraction images from arbitrary regions of the GeoTIFF, we add tiling to the image. Since we must write the tiled image before deleting the untiled version, we effectively double our disk space usage at this step.

In [ ]:
gdal_command = ['C:\\Anaconda\\envs\\py35\\Lib\\site-packages\\osgeo\\gdal_translate',
            '-co', 'TILED=YES',
            os.path.join(tmp_dir, '{}.tif'.format(filename_base)),
            os.path.join(tmp_dir, '{}_tiled.tif'.format(filename_base))]
results = run(gdal_command)
os.remove(os.path.join(tmp_dir, '{}.tif'.format(filename_base)))

naip_filepath = os.path.join(tmp_dir, '{}_tiled.tif'.format(filename_base))

<a name="box"></a>
### Find a lat-lon bounding box for the region

We now find the lat-lon bounding box of the NAIP GeoTIFF. Some locations inside this bounding box will not have any available data, but the box is still useful for first-order estimation of the usable region.

In [ ]:
from osgeo import gdal
from gdalconst import *
import osr
from mpl_toolkits.basemap import Basemap
from collections import namedtuple

LatLonBounds = namedtuple('LatLonBounds', ['llcrnrlat', 'llcrnrlon', 'urcrnrlat', 'urcrnrlon'])

def get_region_bounds(naip_filepath):
    ''' Finds a bounding box for the NAIP GeoTIFF in lat/lon '''
    naip_image = gdal.Open(naip_filepath, GA_ReadOnly)
    naip_proj = osr.SpatialReference()
    naip_proj.ImportFromWkt(naip_image.GetProjection())
    naip_ulcrnrx, naip_xstep, _, naip_ulcrnry, _, naip_ystep = naip_image.GetGeoTransform()

    world_map = Basemap(lat_0 = 0,
                        lon_0 = 0,
                        llcrnrlat=-90, urcrnrlat=90,
                        llcrnrlon=-180, urcrnrlon=180,
                        resolution='c', projection='stere')
    world_proj = osr.SpatialReference()
    world_proj.ImportFromProj4(world_map.proj4string)
    ct_to_world = osr.CoordinateTransformation(naip_proj, world_proj)
    
    lats = []
    lons = []
    for corner_x, corner_y in [(naip_ulcrnrx, naip_ulcrnry),
                               (naip_ulcrnrx, naip_ulcrnry + naip_image.RasterYSize * naip_ystep),
                               (naip_ulcrnrx + naip_image.RasterXSize * naip_xstep,
                                naip_ulcrnry + naip_image.RasterYSize * naip_ystep),
                               (naip_ulcrnrx + naip_image.RasterXSize * naip_xstep, naip_ulcrnry)]:
        xpos, ypos, _ = ct_to_world.TransformPoint(corner_x, corner_y)
        lon, lat = world_map(xpos, ypos, inverse=True)
        lats.append(lat)
        lons.append(lon)

    return(LatLonBounds(llcrnrlat=min(lats),
                        llcrnrlon=min(lons),
                        urcrnrlat=max(lats),
                        urcrnrlon=max(lons)))

region_bounds = get_bounding_box(naip_filepath)

<a name="meters"></a>
### Get the approximate dimensions of the bounding box in meters

Here we approximate the width/height of the bounding box in meters, using the fact that latitude does not change substantially on the county scale.

In [ ]:
RegionSize = namedtuple('RegionSize', ['width', 'height'])  # in meters!
import numpy as np

def get_approx_region_size(region_bounds):
    ''' Returns the region width (at mid-lat) and height in meters'''
    mid_lat_radians = (region_bounds.llcrnrlat + region_bounds.urcrnrlat) * \
                      (np.pi / 360)
    earth_circumference = 6.371E6 * 2 * np.pi # in meters
    region_middle_width_meters = (region_bounds.urcrnrlon - region_bounds.llcrnrlon) * \
                                 earth_circumference * np.cos(mid_lat_radians) / (360)
    region_height_meters = (region_bounds.urcrnrlat - region_bounds.llcrnrlat) * \
                           earth_circumference / (360)
    return(RegionSize(region_middle_width_meters, region_height_meters))

approx_region_size = get_approx_region_size(region_bounds)

<a name="consistent"></a>
### Finding tiles with consistent land use class

We will extract images of small regions ("tiles") and corresponding labels to use in training or evaluation. ResNet DNNs are traditionally trained with 224 px x 224 px images: this sets our minimum tile size at 224 meters x 224 meters (because our aerial imagery has approx. 1 meter resolution).

Tiles that lie at the boundary of two or more land uses -- e.g., the edge of a forest or a shoreline -- may cause confusion during model training and evaluation. We chose tiles that are "reasonably homeogeneous" in land use class by confirming that land use type is consistent at nine regularly-spaced interior points. (We found through experimentation that requiring complete homogeneity limited our dataset size and increased the processing time substantially.)

#### Define helper functions

The function below defines a point labeler that we will use for tile selection. At the same time, we define an image extractor that we will use later in the tutorial. (Both functions require a coordinate transform from lat-lon to GeoTIFF-specific coordinates; defining them together keeps things succinct.)

In [ ]:
from PIL import Image
import fiona
from shapely.geometry import shape

def create_helper_functions(region_bounds, nlcd_filepath, naip_filepath):
    ''' Makes helper functions to label points (NLCD) and extract tiles (NAIP) '''
    nlcd_image = gdal.Open(nlcd_filepath, GA_ReadOnly)
    nlcd_proj = osr.SpatialReference()
    nlcd_proj.ImportFromWkt(nlcd_image.GetProjection())
    nlcd_ulcrnrx, nlcd_xstep, _, nlcd_ulcrnry, _, nlcd_ystep = nlcd_image.GetGeoTransform()
    
    naip_image = gdal.Open(naip_filepath, GA_ReadOnly)
    naip_proj = osr.SpatialReference()
    naip_proj.ImportFromWkt(naip_image.GetProjection())
    naip_ulcrnrx, naip_xstep, _, naip_ulcrnry, _, naip_ystep = naip_image.GetGeoTransform()
    
    region_map = Basemap(lat_0 = (region_bounds.llcrnrlat + region_bounds.urcrnrlat)/2,
                         lon_0 = (region_bounds.llcrnrlon + region_bounds.urcrnrlon)/2,
                         llcrnrlat=region_bounds.llcrnrlat,
                         llcrnrlon=region_bounds.llcrnrlon,
                         urcrnrlat=region_bounds.urcrnrlat,
                         urcrnrlon=region_bounds.urcrnrlon,
                         resolution='c',
                         projection='stere')
    
    region_proj = osr.SpatialReference()
    region_proj.ImportFromProj4(region_map.proj4string)
    ct_to_nlcd = osr.CoordinateTransformation(region_proj, nlcd_proj)
    ct_to_naip = osr.CoordinateTransformation(region_proj, naip_proj)
    
    region_shapes = []
    for i in fiona.open('{}.shp'.format(naip_filepath.split('_tiled.')[0])):
        region_shapes.append(shape(i['geometry']))

    def get_nlcd_label(point):
        ''' Project lat/lon point to NLCD GeoTIFF; return label of that point '''
        basemap_coords = region_map(point.lon, point.lat)  # NB unusual argument order
        x, y, _ = [int(i) for i in ct_to_nlcd.TransformPoint(*basemap_coords)]
        xoff = int(round((x - nlcd_ulcrnrx) / nlcd_xstep))
        yoff = int(round((y - nlcd_ulcrnry) / nlcd_ystep))
        label = int(nlcd_image.ReadAsArray(xoff=xoff, yoff=yoff, xsize=1, ysize=1))
        return(label)
    
    def complete_intersection(xx, yy):
        ''' Check whether a tile lies completely within the region's shapefile '''
        tile_shape = shape({'type': 'Polygon',
                            'coordinates': [[(xx[0, 0], yy[0, 0]),
                                             (xx[-1, 0], yy[-1, 0]),
                                             (xx[-1, -1], yy[-1, -1]),
                                             (xx[0, -1], yy[0, -1])]]})
        tile_area = tile_shape.area
        intersection_area = 0.0
        for region_shape in region_shapes:
            if region_shape.intersects(tile_shape):
                intersection_area += region_shape.intersection(tile_shape).area
        if abs(tile_area - intersection_area) < 1.0:
            return(True)
        else:
            return(False)
        
    def get_naip_tile(tile_bounds, tile_size):
        ''' Check that tile lies within county bounds; if so, extract its image '''
        
        # Transform tile bounds in lat/lon to NAIP projection coordinates
        xmax, ymax = region_map(tile_bounds.urcrnrlon, tile_bounds.urcrnrlat)
        xmin, ymin = region_map(tile_bounds.llcrnrlon, tile_bounds.llcrnrlat)
        xstep = (xmax - xmin) / tile_size.width
        ystep = (ymax - ymin) / tile_size.height

        grid = np.mgrid[xmin:xmax:tile_size.width * 1j, ymin:ymax:tile_size.height * 1j]
        shape = grid[0, :, :].shape
        size = grid[0, :, :].size
        xy_target = np.array(ct_to_naip.TransformPoints(grid.reshape(2, size).T))
        xx = xy_target[:,0].reshape(shape)
        yy = xy_target[:,1].reshape(shape)
        
        # Check that the requested tile does not lie at the region boundary
        if not complete_intersection(xx, yy):
            return(None)
        
        # Extract rectangle from NAIP GeoTIFF containing superset of needed points
        xoff = int(round((xx.min() - naip_ulcrnrx) / naip_xstep))
        yoff = int(round((yy.max() - naip_ulcrnry) / naip_ystep))
        xsize_to_use = int(np.ceil((xx.max() - xx.min())/np.abs(naip_xstep))) + 1
        ysize_to_use = int(np.ceil((yy.max() - yy.min())/np.abs(naip_ystep))) + 1
        data = naip_image.ReadAsArray(xoff=xoff,
                                      yoff=yoff,
                                      xsize=xsize_to_use,
                                      ysize=ysize_to_use)        
        # Map the pixels of interest in NAIP GeoTIFF to the tile (might involve rotation or scaling)
        image = np.zeros((xx.shape[1], xx.shape[0], 3)).astype(int)  # rows are height, cols are width, third dim is color
        
        try:
            for i in range(xx.shape[0]):
                for j in range(xx.shape[1]):
                    x_idx = int(round((xx[i,j] - naip_ulcrnrx) / naip_xstep)) - xoff
                    y_idx = int(round((yy[i,j] - naip_ulcrnry) / naip_ystep)) - yoff
                    image[xx.shape[1] - j - 1, i, :] = data[:, y_idx, x_idx]
        except TypeError as e:
            # The following can occur if our pixel superset request exceeds the GeoTIFF's bounds
            return(None)
                
        image = Image.fromarray(image.astype('uint8'))
        return(image)
    
    return(get_nlcd_label, get_naip_tile)
get_nlcd_label, get_naip_tile = create_helper_functions(region_bounds, nlcd_filepath, naip_filepath)

#### Tile selection

We form a rectangular grid of candidate tiles spanning as much of the (approximated-as-rectangular) bounding box as possible. Tiles that have "reasonably homogeneous" land use classification are retained for image extraction in the next step. 

In [ ]:
LatLonPosition = namedtuple('LatLonPosition', ['lat', 'lon'])
Tile = namedtuple('Tile', ['bounds', 'label'])

def find_tiles_with_consistent_labels(region_bounds, region_size, tile_size):
    ''' Find tiles for which nine grid points all have the same label '''
    tiles_wide = int(np.floor(region_size.width / tile_size.width))
    tiles_tall = int(np.floor(region_size.height / tile_size.height))
    tile_width = (region_bounds.urcrnrlon - region_bounds.llcrnrlon) / (region_size.width / tile_size.width)
    tile_height = (region_bounds.urcrnrlat - region_bounds.llcrnrlat) / (region_size.height / tile_size.height)
    
    current_lat = region_bounds.llcrnrlat
    current_lon = region_bounds.llcrnrlon
    
    tiles_to_use = []
    for i in range(tiles_tall):
        for j in range(tiles_wide):
            try:
                labels = []
                for k in range(3):
                    for ell in range(3):
                        labels.append(get_nlcd_label(LatLonPosition(lat=current_lat + tile_width * (1 + 2*k) / 6,
                                                                    lon=current_lon + tile_height * (1 + 2*ell) / 6)))
                num_matching = np.sum(np.array(labels) == labels[4])
                if (num_matching == 9) and (labels[4] > 10):
                    bounds = LatLonBounds(llcrnrlat=current_lat,
                                          llcrnrlon=current_lon,
                                          urcrnrlat=current_lat + tile_height,
                                          urcrnrlon=current_lon + tile_width)
                    tiles_to_use.append(Tile(bounds=bounds,
                                             label=labels[4]))
            except KeyError:
                pass
            current_lon += tile_width
        current_lon = region_bounds.llcrnrlon
        current_lat += tile_height
    return(tiles_to_use)

tiles = find_tiles_with_consistent_labels(region_bounds, approx_region_size, tile_size)

<a name="extraction"></a>
### Extract and save images for tiles of interest

The most time-consuming step is the extraction of images from the GeoTIFF. We use a helper function defined earlier, `naip_get_tile`, to check that the tile lies entirely inside the county shapefile and, if so, return the extracted image. Images are sorted into directories based on land use class.

In [ ]:
import uuid

def extract_tiles(tiles, dest_folder):
    ''' Coordinates saving tile data, including extracted images and CSV descriptions '''
    my_region_id = uuid.uuid4()
    if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
    tile_descriptions = []
    for i, tile in enumerate(tiles):
        tile_image = get_naip_tile(tile.bounds, tile_size)
        if (tile_image is None):
            continue  # tile did not lie entirely within the county boundary (at least partially blank)
        my_directory = os.path.join(dest_folder, '{:02d}'.format(tile.label))
        my_filename = os.path.join(my_directory, '{}.png'.format(i))
        if not os.path.exists(my_directory):
            os.makedirs(my_directory)
        tile_image.save(my_filename, 'PNG')
    return

extract_tiles(tiles, os.path.join(tmp_dir, filename_base))